Basic

In [ ]:
def load_data(datapath, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
    ])
    dataset = ImageFolder(root=datapath, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

In [ ]:
class CustomFaceNet(torch.nn.Module):
    def __init__(self, num_classes=1, freeze_layers=0):
        super(CustomFaceNet, self).__init__()
        self.model = InceptionResnetV1(pretrained='vggface2')

        if freeze_layers > 0:
            for param in list(self.model.parameters())[:freeze_layers]:
                param.requires_grad = False

        self.fc = torch.nn.Linear(512, num_classes)

    def forward(self, x):
        return self.fc(self.model(x))

In [ ]:
def finetune_facenet(data_loader, num_epochs=5, freeze_layers=0):
    model = CustomFaceNet().to(device)

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

    model.train()
    for epoch in range(num_epochs):
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            labels = labels.float().unsqueeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

In [ ]:
def test_model(model, test_images_folder):
    model.eval()
    results = {}

    for img_name in os.listdir(test_images_folder):
        img_path = os.path.join(test_images_folder, img_name)
        image = Image.open(img_path).convert('RGB')

        transform = transforms.Compose([
            transforms.Resize((160, 160)),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            predicted_prob = torch.sigmoid(output).item()
            results[img_name] = 'M' if predicted_prob > 0.5 else 'F'

    return results

In [ ]:
def main():
    data_folder = '/path/to/dataset/'
    test_images_folder = '/path/to/test_images/'

    data_loader = load_data(data_folder)

    results = {}
    for freeze_layers in [0, 5, 10, 15]:
        model = finetune_facenet(data_loader, num_epochs=5, freeze_layers=freeze_layers)
        results[freeze_layers] = test_model(model, test_images_folder)

    with open('results.json', 'w') as json_file:
        json.dump(results, json_file)

if __name__ == "__main__":
    main()

Advanced

In [ ]:
def load_data(datapath, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),  # Resize images to a consistent size
        transforms.ToTensor(),          # Convert images to PyTorch tensors
    ])
    dataset = datasets.ImageFolder(root=datapath, transform=transform)  # Automatically labels based on folder names
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  # Load data in batches, shuffle for randomness
    return loader

In [ ]:
class CustomFaceNet(nn.Module):
    def __init__(self, num_classes=1):
        super(CustomFaceNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # First conv layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Second conv layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # Pooling to reduce dimensions
        self.fc1 = nn.Linear(64 * 40 * 40, 512)  # Fully connected layer
        self.fc2 = nn.Linear(512, num_classes)  # Output layer
        self.sigmoid = nn.Sigmoid()  # Activation for binary classification

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # Apply ReLU after first conv and pool
        x = self.pool(nn.ReLU()(self.conv2(x)))  # Apply ReLU after second conv and pool
        x = x.view(-1, 64 * 40 * 40)  # Flatten the feature map
        x = nn.ReLU()(self.fc1(x))  # Apply ReLU after the first fully connected layer
        x = self.sigmoid(self.fc2(x))  # Sigmoid for binary output
        return x

In [ ]:
def finetune_facenet(data_loader, num_epochs=5):
    model = CustomFaceNet().to(device)  # Initialize the model and move to device
    criterion = nn.BCELoss()  # Loss function for binary classification
    optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Optimizer for training

    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to device
            optimizer.zero_grad()  # Zero the gradient buffers
            outputs = model(images)  # Forward pass
            labels = labels.float().unsqueeze(1)  # Reshape labels for loss computation
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagate
            optimizer.step()  # Update weights
    return model

In [ ]:
def test_model(model, test_images_folder):
    model.eval()
    results = {}

    for img_name in os.listdir(test_images_folder):
        img_path = os.path.join(test_images_folder, img_name)
        image = Image.open(img_path).convert('RGB')

        transform = transforms.Compose([
            transforms.Resize((160, 160)),
            transforms.ToTensor(),
        ])
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            predicted_prob = output.item()
            results[img_name] = {
                'label': 'M' if predicted_prob > 0.5 else 'F',
                'predicted_prob': predicted_prob
            }

    return results


In [ ]:
def main():
    data_folder = '/path/to/dataset/'  # Path to training data
    test_images_folder = '/path/to/test_images/'  # Path to test data

    data_loader = load_data(data_folder)  # Load training data

    model = finetune_facenet(data_loader, num_epochs=5)  # Train the model
    results = test_model(model, test_images_folder)  # Test the model

    with open('results_advance.json', 'w') as json_file:
        json.dump(results, json_file)  # Save results to a JSON file

if __name__ == "__main__":
    main()